In [1]:
import pandas as pd
import numpy as np
import csv
import pickle

In [2]:
sample  = pickle.load( open( "data/twitter_data_clean.p", "rb" ) )
sample.head()

,city,country,day,followersCount,friendsCount,hour,id,inReplyTo,latitude,longitude,month,placeLatitude,placeLongitude,state,statusesCount,text,userId,userLocation,week_number,year
551,Binningen,ch,9.0,14249,9260.0,18.0,10231423626,NaN,47.5455,7.58531,3.0,47.5367,7.57849,Basel-Landschaft,19585.0,"The new apartment is nice, but there is no Wif...",6.25728e+06,"Potsdam, Germany",10.0,2010.0
609,Zürich,ch,10.0,177,136.0,22.0,10292646240,NaN,47.3876,8.52725,3.0,47.3791,8.50021,Zürich,5167.0,Is that wet yet solid stuff on my screen suppo...,1.5602e+07,"Zürich, Switzerland",10.0,2010.0
611,Genève,ch,11.0,471,82.0,5.0,10309829732,NaN,46.2006,6.13183,3.0,46.1996,6.13011,Genève,3363.0,I'm at DCTI - David Dufour in Geneva http://go...,625553,"Geneva, Switzerland",10.0,2010.0
612,Köniz,ch,11.0,586,508.0,6.0,10310391132,NaN,46.8957,7.44235,3.0,46.9214,7.38855,Bern - Berne,9016.0,God morgon! :-),1.7341e+07,"Bern, Switzerland",10.0,2010.0
618,Genève,ch,11.0,2230,387.0,7.0,10311568050,NaN,46.2043,6.19900,3.0,46.1938,6.15415,Genève,10605.0,"At this very minute, the sun is pink.",634553,"Geneva, Switzerland",10.0,2010.0


In [3]:
sample.isnull().sum()

city                 88225
country                 24
day                      1
followersCount         148
friendsCount           161
hour                     0
id                       0
inReplyTo         11255929
latitude           5760885
longitude          5760885
month                    1
placeLatitude            0
placeLongitude           0
state                  274
statusesCount          167
text                149864
userId                   0
userLocation       4041308
week_number              1
year                     1
dtype: int64

In [6]:
sample.astype

<bound method NDFrame.astype of                       city country   day followersCount  friendsCount  hour  \
551              Binningen      ch   9.0          14249        9260.0  18.0   
609                 Zürich      ch  10.0            177         136.0  22.0   
611                 Genève      ch  11.0            471          82.0   5.0   
612                  Köniz      ch  11.0            586         508.0   6.0   
618                 Genève      ch  11.0           2230         387.0   7.0   
622                 Genève      ch  11.0            167         277.0  11.0   
623                  Basel      ch  11.0          14249        9260.0  11.0   
630                 Zürich      ch  11.0            230         276.0  13.0   
632       Birmensdorf (ZH)      ch  11.0            533         292.0  15.0   
633                 Zürich      ch  11.0            394         266.0  16.0   
634                 Zürich      ch  11.0             75         161.0  17.0   
635                 

A few cities and states is not found, so we can drop it.

In [7]:
user_all=sample.dropna(subset=['state'])
user_all.isnull().sum()

city                 88197
country                  0
day                      1
followersCount         148
friendsCount           161
hour                     0
id                       0
inReplyTo         11255702
latitude           5760885
longitude          5760885
month                    1
placeLatitude            0
placeLongitude           0
state                    0
statusesCount          167
text                149862
userId                   0
userLocation       4041219
week_number              1
year                     1
dtype: int64

In [8]:
user=user_all.loc[user_all['country']=='ch']
user.shape

(10129732, 20)

# Cleaning unusual trajectories

Revocer time for calculating valid user in reasonable moving speed.

In [9]:
import time
import datetime

t_count=user[['year','month','day','hour']]
temp=pd.to_datetime(t_count)
temp.head()
user['time']=temp
user.head()

D:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,city,country,day,followersCount,friendsCount,hour,id,inReplyTo,latitude,longitude,...,placeLatitude,placeLongitude,state,statusesCount,text,userId,userLocation,week_number,year,time
551,Binningen,ch,9.0,14249,9260.0,18.0,10231423626,NaN,47.5455,7.58531,...,47.5367,7.57849,Basel-Landschaft,19585.0,"The new apartment is nice, but there is no Wif...",6.25728e+06,"Potsdam, Germany",10.0,2010.0,2010-03-09 18:00:00
609,Zürich,ch,10.0,177,136.0,22.0,10292646240,NaN,47.3876,8.52725,...,47.3791,8.50021,Zürich,5167.0,Is that wet yet solid stuff on my screen suppo...,1.5602e+07,"Zürich, Switzerland",10.0,2010.0,2010-03-10 22:00:00
611,Genève,ch,11.0,471,82.0,5.0,10309829732,NaN,46.2006,6.13183,...,46.1996,6.13011,Genève,3363.0,I'm at DCTI - David Dufour in Geneva http://go...,625553,"Geneva, Switzerland",10.0,2010.0,2010-03-11 05:00:00
612,Köniz,ch,11.0,586,508.0,6.0,10310391132,NaN,46.8957,7.44235,...,46.9214,7.38855,Bern - Berne,9016.0,God morgon! :-),1.7341e+07,"Bern, Switzerland",10.0,2010.0,2010-03-11 06:00:00
618,Genève,ch,11.0,2230,387.0,7.0,10311568050,NaN,46.2043,6.19900,...,46.1938,6.15415,Genève,10605.0,"At this very minute, the sun is pink.",634553,"Geneva, Switzerland",10.0,2010.0,2010-03-11 07:00:00


In [10]:
# velticity calculation, miles per hour
from geopy.distance import vincenty 

def speed(loc1, loc2, t1, t2):
    d = vincenty(loc1,loc2).miles
    dt = t2-t1
    dh = dt.seconds/3600.0 #to_hours
    if dh :
        return d/dh 
    else: #within one hour, assume it travels within half an hour
        return d/0.5

In [72]:
#ref: Exploring Multi-Scale Spatiotemporal Twitter User Mobility Patterns with a Visual-Analytics Approach
#Space-Time Twitter User Trajectories
#To remove non-human users based on unusual relocating speed: e.g. speed (loc_i − loc_i−1) > 240 m/s

#Travelling in Switzerland mainly takes train(despite some ones take flight)
#Data are in time-order, so the location is continuously changed
#Since we clean the data before, time is accurate to hour, so we set a loose thedshold like 200 mils/h 


#remove non-human user
def human_user(c_tuple, speed_thed=200):
    lon,lan,t = tuple(zip(*c_tuple))
    point = list(zip(lan,lon))
    normal=[True]*(len(c_tuple))
    
    if len(c_tuple) == 1:
        normal = [False]*(len(c_tuple)) # remove only one tweet
    if len(c_tuple) == 2 :
        v = speed(point[0],point[1],t[0],t[1])
        if v > speed_thed:
            normal = [False]*(len(c_tuple))
    else:
        loc1 = point[0:-2]
        loc2 = point[1:-1]
        loc3 = point[2::]
        #Using geopy's vincenty to find distances in pandas data frame
        for i in range(len(loc1)):
            v1_2=speed(loc2[i],loc1[i],t[i],t[i+1])
            v2_3=speed(loc2[i],loc3[i],t[i+1],t[i+2])
            v1_3=speed(loc1[i],loc3[i],t[i],t[i+2])
        
            # Check if loc1 is a normal location with reasonable speed
            # If there is something wrong with loc1->loc2, we check loc2->loc3 & loc1->loc3 to ensure if loc2/3 is wrong
        
            if v1_2 > speed_thed: # loc1->loc2 innormal
                if v2_3 > speed_thed: # loc2->loc1/3 innormal -> loc2 innormal
                    normal[i+1]=False 
                    if v1_3 > speed_thed: # loc1->loc2/3 innormal -> loc1 innormal
                        normal[i]=False 
                        if i == len(loc1)-1: # last index with all wrong
                            normal[i+2]=False
                else: # loc1->loc2 innormal, loc2->loc1/3 normal -> loc2 normal, loc1 innormal
                    normal[i]=False
            else: # loc1->loc2 normal
                if v2_3 > speed_thed:
                    if v1_3 > speed_thed: # loc3->loc1/2 innormal -> loc3 innormal
                        normal[i+2]=False
                    else:
                        if (i == len(loc1)-1): # last index, just loc2->loc3 innormal -> loc2 innormal
                            normal[i+1]=False
        
    return normal

In [73]:
#twitter_user=twitter_data.groupby(twitter_data['userId'])
#.order(ascending=True)
user_trace=['userId','year','month','week_number','day']
user['temp'] = tuple(zip(user['placeLatitude'],user['placeLongitude'],user['time']))
normal = user.groupby(by=user_trace)['temp'].transform(lambda x: human_user(x))

D:\anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
normal.shape

(10129732,)

In [75]:
user.shape

(10129732, 22)

In [76]:
normal_user = user[normal].reset_index(drop=True)
normal_user.shape

KeyError: '[False False False ..., False True True] not in index'

In [16]:
user.to_csv("ch_user.csv", sep='\t', encoding='utf-8')

In [17]:
with open ( "judge.p", "wb" ) as output2:
    pickle.dump(normal, output2, pickle.HIGHEST_PROTOCOL)

In [77]:
n_split=100
n_user=np.array_split(user, n_split)
n_judge=np.array_split(normal, n_split)

In [78]:
def split_judge(user,judge,r1,r2):
    frames=[]
    for i in range(r1,r2):
        df=n_user[i][n_judge[i]]
        frames.append(df)
    return frames

In [80]:
clean_data[0] = split_judge(n_user[0:10],n_judge[0:10],0,10)
#clean_data[1] = split_judge(n_user[11:20],n_judge[11:20],11,20)
clean_data[2] = split_judge(n_user[21:30],n_judge[21:30],21,30)
clean_data[3] = split_judge(n_user[31:40],n_judge[31:40],31,40)
clean_data[4] = split_judge(n_user[41:50],n_judge[41:50],41,50)
clean_data[5] = split_judge(n_user[51:60],n_judge[51:60],51,60)
clean_data[6] = split_judge(n_user[61:70],n_judge[61:70],61,70)
clean_data[7] = split_judge(n_user[71:80],n_judge[71:80],71,80)
clean_data[8] = split_judge(n_user[81:90],n_judge[81:90],81,90)
clean_data[9] = split_judge(n_user[91:100],n_judge[91:100],91,100)

In [63]:

for n in range(10):
    frames=[]
    for i in range(10):
        print(i)
        df=n_user[n*10+i][n_judge[n*10+i]]
        frames.append(df)
    #normal_user = pd.concat([normal_user,frames])
    clean_data[n] = pd.concat(frames)
    #clean_data.append(frames)

normal_user=clean_data[0]
for n in range(1,10):
    normal_user = pd.concat([normal_user,clean_data[n]])
#normal_user = pd.concat(frames)
normal_user.shape

#n_user[0][n_judge[0]]

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8


KeyError: '[True False False ..., True True True] not in index'

In [90]:
(n_judge[18].index==n_user[18].index).sum()

101298

In [91]:
n_split=100
n_user1=np.array_split(n_user[18], n_split)
n_judge1=np.array_split(n_judge[18], n_split)

In [93]:
for n in range(10):
    frames=[]
    for i in range(10):
        print(n*10+i)
        df=n_user1[n*10+i][n_judge1[n*10+i]]
        frames.append(df)
    #normal_user = pd.concat([normal_user,frames])
    #clean_data[n] = pd.concat(frames)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


KeyError: '[True True True ..., True True True] not in index'

In [97]:
n_split=100
n_user2=np.array_split(n_user1[81], n_split)
n_judge2=np.array_split(n_judge1[81], n_split)
for n in range(10):
    frames=[]
    for i in range(10):
        print(n*10+i)
        df=n_user2[n*10+i][n_judge2[n*10+i]]
        frames.append(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


KeyError: '[False True False True True True True True True\n (46.232199999999999, 6.0789999999999997, NaT)] not in index'

# Store trace

Next,we hope to define daily trace of every user.

In [ ]:
# Since tweets are in time order, we can just store daily trace by apprending locations(canton)

def daily_trace(r_location):
    start=r_location.values[0:-1]
    end=r_location.values[1::]
    index=(start!=end)
    trace=np.nan
    
    if index.any(): # Remove inner-canton trace 
        trace='->'.join(start[index])
        trace='->'.join([trace,end[index][-1]])
        
    return trace    
    

In [ ]:
user_dtrace = normal_user.groupby(by=user_trace)['state'].apply(lambda x: daily_trace(x))


In [ ]:
user_dtrace=user_dtrace.dropna()
user_dtrace.head()

In [ ]:
with open ( "state_mov_all.pkl", "wb" ) as output:
    pickle.dump(temp, output, pickle.HIGHEST_PROTOCOL)